# Converts gridded data to parquet format

In [1]:
import pandas as pd
from pathlib import Path
import geopandas as gp
import re
gp.options.io_engine = "pyogrio"
from shapely.geometry import Point, Polygon
from datetime import datetime
import plotly.express as px
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape as geoshape
from shapely.geometry import Polygon
import numpy as np
from joblib import Parallel, delayed
from scipy.spatial import cKDTree
import netCDF4
import xarray as xr
import pyproj
from shapely.ops import transform
from typing import List

In [2]:
ws_dir = Path("/home/aschneuwl/workspace/")
ws2_dir = Path("/mnt/wks3/aschneuwl/workspace/data/preprocessed/")

In [3]:
def gridded_to_df(fpath: Path) -> pd.DataFrame:
        # Converts gridded data into pandas DataFrame
        df = xr.open_dataset(fpath).to_dataframe().reset_index().dropna()
        #df[""] = df.apply(lambda x: Point(x.lon, x.lat), axis=1)
        return df

def gridded_data_files_to_parquet(dir: str, measure: str, dst_dir: Path) -> pd.DataFrame:
    gridded_data_dir = Path(dir)
    files = gridded_data_dir.glob(f"{measure}_*.nc")
    
    print("Load")
    gridded_dfs = Parallel(n_jobs=-1, verbose=10)(delayed(gridded_to_df)(fpath) for fpath in files)
    gridded_df = pd.concat(gridded_dfs)
    gridded_df["year"] = gridded_df.time.dt.year
    print("Store...")
    gridded_df.to_parquet(dst_dir / Path(f"{measure}.parquet"), partition_cols=["year"])

# Daily Minimum Temperature

In [ ]:
gridded_data_files_to_parquet("/home/aschneuwl/workspace/data/meteo/meteosuisse/grid/NEW_CH01_ALL-DATA-v2024/", "TminD", ws2_dir / Path("meteo"))

# Daily Maximum Temperature

In [ ]:
gridded_data_files_to_parquet("/home/aschneuwl/workspace/data/meteo/meteosuisse/grid/NEW_CH01_ALL-DATA-v2024/", "TmaxD", ws2_dir / Path("meteo"))

# Daily Average Temperature

In [4]:
gridded_data_files_to_parquet("/home/aschneuwl/workspace/data/meteo/meteosuisse/grid/NEW_CH01_ALL-DATA-v2024/", "TabsD", ws2_dir / Path("meteo"))

Load


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  63 | elapsed:   30.7s remaining:  4.1min
[Parallel(n_jobs=-1)]: Done  14 out of  63 | elapsed:   44.3s remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  21 out of  63 | elapsed:   57.9s remaining:  1.9min
[Parallel(n_jobs=-1)]: Done  28 out of  63 | elapsed:  1.2min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  35 out of  63 | elapsed:  1.4min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  42 out of  63 | elapsed:  1.6min remaining:   49.1s
[Parallel(n_jobs=-1)]: Done  49 out of  63 | elapsed:  1.9min remaining:   31.8s
[Parallel(n_jobs=-1)]: Done  56 out of  63 | elapsed:  2.1min remaining:   15.5s
[Parallel(n_jobs=-1)]: Done  63 out of  63 | elapsed:  2.3min finished


Store...


# Daily Precipitation

In [ ]:
gridded_data_files_to_parquet("/home/aschneuwl/workspace/data/meteo/meteosuisse/grid/NEW_CH01_ALL-DATA-v2024/", "RhiresD", ws2_dir / Path("meteo"))

# Daily Sunshine Duration

In [ ]:
gridded_data_files_to_parquet("/home/aschneuwl/workspace/data/meteo/meteosuisse/grid/NEW_CH01_ALL-DATA-v2024/", "SrelD", ws2_dir / Path("meteo"))

# Plotting

## Daily Maximum Temperature

In [ ]:
localities_switzerland_fpath = ws_dir / Path("agecon-thesis/notebooks/00_preprocessing/swiss_localities_with_altitudes.parquet")
localities_swiss = gp.read_parquet(localities_switzerland_fpath)

In [ ]:
localities_swiss = gp.read_file("/home/aschneuwl/workspace/agecon-thesis/notebooks/AMTOVZ_ZIP.shp")

lv95 = pyproj.CRS('EPSG:2056')
wgs84 = pyproj.CRS('EPSG:4326')

fig = px.choropleth_mapbox(zip_max_temp, geojson=localities_swiss, featureidkey="properties.ZIP_ID",
                           locations='ZIP_ID', 
                           color='TmaxD',
                           mapbox_style="carto-positron",
                           zoom=6.3, 
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.5,
                           width=1000,
                           height=750,
                           hover_data=["TmaxD"],
                           labels={"TmaxD": "Max Daily Temperature", "ZIP4": "Locality"}
                          )
fig.show()